In [37]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import os

In [15]:
df = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")
df

/tmp/ipykernel_99059/203287006.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230.0,40.621790,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,09/21/2022,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212907,10/09/2016,9:30,STATEN ISLAND,10305.0,40.612595,-74.071655,"(40.612595, -74.071655)",NaN,NaN,254 CLIFTON AVENUE,...,NaN,NaN,NaN,NaN,3536789,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
2212908,09/20/2016,12:30,BROOKLYN,11229.0,40.610725,-73.955574,"(40.610725, -73.955574)",NaN,NaN,1804 AVENUE P,...,Unspecified,NaN,NaN,NaN,3527277,Sedan,Sedan,NaN,NaN,NaN
2212909,09/19/2016,13:00,NaN,NaN,40.611767,-73.976380,"(40.611767, -73.97638)",24 AVENUE,NaN,NaN,...,NaN,NaN,NaN,NaN,3523434,Sedan,NaN,NaN,NaN,NaN
2212910,09/28/2016,11:20,NaN,NaN,40.620560,-73.989440,"(40.62056, -73.98944)",63 STREET,NaN,NaN,...,Unspecified,NaN,NaN,NaN,3529408,Station Wagon/Sport Utility Vehicle,Pick-up Truck,NaN,NaN,NaN


In [ ]:
df.isnull().sum().sum() # Muitos valores nulos impossivel exclui-los

np.int64(18922504)

In [ ]:
df.isna().sum() # colunas off street name, contributing factor 3 veicle ate o 5 , vehicle type code 3 ate o 5 sao colunas com muitos valores nulos e podem ser removidas
missing_percent = df.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))
df_removed = df.drop(columns=['OFF STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])
df_removed.isna().sum().sum()

VEHICLE TYPE CODE 5              99.561799
CONTRIBUTING FACTOR VEHICLE 5    99.547564
VEHICLE TYPE CODE 4              98.408116
CONTRIBUTING FACTOR VEHICLE 4    98.348556
VEHICLE TYPE CODE 3              93.045860
CONTRIBUTING FACTOR VEHICLE 3    92.765325
OFF STREET NAME                  82.421352
CROSS STREET NAME                38.204095
ZIP CODE                         30.683687
BOROUGH                          30.671260
ON STREET NAME                   21.752966
VEHICLE TYPE CODE 2              20.007935
CONTRIBUTING FACTOR VEHICLE 2    16.022553
LOCATION                         10.858679
LATITUDE                         10.858679
LONGITUDE                        10.858679
VEHICLE TYPE CODE 1               0.723797
CONTRIBUTING FACTOR VEHICLE 1     0.352070
NUMBER OF PERSONS KILLED          0.001401
NUMBER OF PERSONS INJURED         0.000813
CRASH DATE                        0.000000
NUMBER OF PEDESTRIANS INJURED     0.000000
CRASH TIME                        0.000000
NUMBER OF M

np.int64(4226587)

In [29]:
print(df.shape)
print(df_removed.shape)

(2212912, 29)
(2212912, 22)


In [34]:
num_cols = df_removed.select_dtypes(include=['number']).columns.to_list()
print(num_cols)
cat_cols = df_removed.select_dtypes(include=['object']).columns.to_list()
print(cat_cols)


['LATITUDE', 'LONGITUDE', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'COLLISION_ID']
['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']


In [40]:
transformers_imput = [
    ('num_imputer', SimpleImputer(strategy='median'),num_cols),
    ('freq_imputer', SimpleImputer(strategy='most_frequent'),cat_cols)
]
ct_imput = ColumnTransformer(transformers_imput, remainder='passthrough')
df_imputed_array = ct_imput.fit_transform(df_removed)

# reconstruir DataFrame com as colunas originais (reordenadas)
df_imputed = pd.DataFrame(df_imputed_array, columns=num_cols + cat_cols)

# garantir tipos corretos
df_imputed[num_cols] = df_imputed[num_cols].apply(pd.to_numeric, errors='coerce')

(2212912, 22)


In [44]:
df_imputed

,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,...,CRASH TIME,BOROUGH,ZIP CODE,LOCATION,ON STREET NAME,CROSS STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,40.720320,-73.926796,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,2:39,BROOKLYN,11207.0,"(0.0, 0.0)",WHITESTONE EXPRESSWAY,20 AVENUE,Aggressive Driving/Road Rage,Unspecified,Sedan,Sedan
1,40.720320,-73.926796,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,11:45,BROOKLYN,11207.0,"(0.0, 0.0)",QUEENSBORO BRIDGE UPPER,3 AVENUE,Pavement Slippery,Unspecified,Sedan,Sedan
2,40.621790,-73.970024,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1:29,BROOKLYN,11230.0,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,Unspecified,Unspecified,Moped,Sedan
3,40.720320,-73.926796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6:55,BROOKLYN,11207.0,"(0.0, 0.0)",THROGS NECK BRIDGE,3 AVENUE,Following Too Closely,Unspecified,Sedan,Pick-up Truck
4,40.720320,-73.926796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13:21,BROOKLYN,11207.0,"(0.0, 0.0)",BROOKLYN BRIDGE,3 AVENUE,Passing Too Closely,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212907,40.612595,-74.071655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9:30,STATEN ISLAND,10305.0,"(40.612595, -74.071655)",BROADWAY,3 AVENUE,Unspecified,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan
2212908,40.610725,-73.955574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12:30,BROOKLYN,11229.0,"(40.610725, -73.955574)",BROADWAY,3 AVENUE,Unspecified,Unspecified,Sedan,Sedan
2212909,40.611767,-73.976380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13:00,BROOKLYN,11207.0,"(40.611767, -73.97638)",24 AVENUE,3 AVENUE,Unspecified,Unspecified,Sedan,Sedan
2212910,40.620560,-73.989440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11:20,BROOKLYN,11207.0,"(40.62056, -73.98944)",63 STREET,3 AVENUE,Driver Inattention/Distraction,Unspecified,Station Wagon/Sport Utility Vehicle,Pick-up Truck
